In [1]:
import datetime
import mariadb
import pandas as pd
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import flask
from PIL import Image
import tensorflow as tf
import numpy as np
import base64
import cv2
from flask import Flask, send_from_directory
import time
from io import BytesIO as _BytesIO

C:\Users\eiker\anaconda3\envs\ds\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\eiker\anaconda3\envs\ds\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\eiker\anaconda3\envs\ds\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\eiker\anaconda3\envs\ds\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood

In [2]:
def draw_rect(image, box):
    y_min = int(max(1, (box[0] * 320)))
    x_min = int(max(1, (box[1] * 320)))
    y_max = int(min(320, (box[2] * 320)))
    x_max = int(min(320, (box[3] * 320)))
    
    # draw a rectangle on the image
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

In [3]:
def pil_to_b64(im, enc_format='png', verbose=False, **kwargs):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """
    t_start = time.time()

    buff = _BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    t_end = time.time()
    if verbose:
        print(f"PIL converted to b64 in {t_end - t_start:.3f} sec")

    return encoded


def numpy_to_b64(np_array, enc_format='png', scalar=True, **kwargs):
    """
    Converts a numpy image into base 64 string for HTML displaying
    :param np_array:
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :param scalar:
    :return:
    """
    # Convert from 0-1 to 0-255
    if scalar:
        np_array = np.uint8(255 * np_array)
    else:
        np_array = np.uint8(np_array)

    im_pil = Image.fromarray(np_array)

    return pil_to_b64(im_pil, enc_format, **kwargs)

def b64_to_numpy(string, to_scalar=True):
    im = b64_to_pil(string)
    np_array = np.asarray(im)

    if to_scalar:
        np_array = np_array / 255.

    return np_array

# Custom Display Components
def Card(children, **kwargs):
    return html.Section(
        children,
        style=_merge({
            'padding': 20,
            'margin': 5,
            'borderRadius': 5,
            'border': 'thin lightgrey solid',

            # Remove possibility to select the text for better UX
            'user-select': 'none',
            '-moz-user-select': 'none',
            '-webkit-user-select': 'none',
            '-ms-user-select': 'none'
        }, kwargs.get('style', {})),
        **_omit(['style'], kwargs)
    )

def NamedInlineRadioItems(name, short, options, val, **kwargs):
    return html.Div(
        id=f'div-{short}',
        style=_merge({
            'display': 'block',
            'margin-bottom': '5px',
            'margin-top': '5px'
        }, kwargs.get('style', {})),
        children=[
            f'{name}:',
            dcc.RadioItems(
                id=f'radio-{short}',
                options=options,
                value=val,
                labelStyle={
                    'display': 'inline-block',
                    'margin-right': '7px',
                    'font-weight': 300
                },
                style={
                    'display': 'inline-block',
                    'margin-left': '7px'
                }
            )
        ],
        **_omit(['style'], kwargs)
    )

# Display utility functions
def _merge(a, b):
    return dict(a, **b)

def _omit(omitted_keys, d):
    return {k: v for k, v in d.items() if k not in omitted_keys}

def InteractiveImagePIL(image_id,
                        image,
                        enc_format='png',
                        display_mode='fixed',
                        dragmode='select',
                        verbose=False,
                        **kwargs):
    if enc_format == 'jpeg':
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        encoded_image = pil_to_b64(image, enc_format=enc_format, verbose=verbose, quality=80)
    else:
        encoded_image = pil_to_b64(image, enc_format=enc_format, verbose=verbose)

    width, height = image.size

    if display_mode.lower() in ['scalable', 'scale']:
        display_height = '{}vw'.format(round(60 * height / width))
    else:
        display_height = '80vh'

    return dcc.Graph(
        id=image_id,
        figure={
            'data': [],
            'layout': {
                'margin': go.Margin(l=40, b=40, t=26, r=10),
                'xaxis': {
                    'range': (0, width),
                    'scaleanchor': 'y',
                    'scaleratio': 1
                },
                'yaxis': {
                    'range': (0, height)
                },
                'images': [{
                    'xref': 'x',
                    'yref': 'y',
                    'x': 0,
                    'y': 0,
                    'yanchor': 'bottom',
                    'sizing': 'stretch',
                    'sizex': width,
                    'sizey': height,
                    'layer': 'below',
                    'source': HTML_IMG_SRC_PARAMETERS + encoded_image,
                }],
                'dragmode': dragmode,
            }
        },
        style=_merge({
            'height': display_height,
            'width': '100%'
        }, kwargs.get('style', {}))

        **_omit(['style'], kwargs)
    )

def b64_to_pil(string):
    decoded = base64.b64decode(string)
    buffer = _BytesIO(decoded)
    im = Image.open(buffer)

    return im








In [ ]:
app = dash.Dash(__name__)
server = app.server

# Get the Temperature and Humidity from the RaspberryPi via SQL/MariaDB 

#db_connection = mariadb.connect(host='192.168.178.34', database='sensor_logs', user='dblogger', password='guevara87',  port=3306)
#temp_data = pd.read_sql('SELECT * FROM data', db_connection)
#temp_data['dateandtime'] = pd.to_datetime(temp_data['dateandtime'])

def get_value(n_inervals):
    db_connection = mariadb.connect(host='192.168.178.34', database='sensor_logs', user='dblogger', password='guevara87',  port=3306)
    temp_data = pd.read_sql('SELECT * FROM data', db_connection)
    temp_data['dateandtime'] = pd.to_datetime(temp_data['dateandtime'])
    return temp_data



# Bannerimage
image_filename = './assets/banner.jpg' 
encoded_image = pil_to_b64(Image.open(image_filename).copy(), enc_format='jpeg')

# Setting variables for the NN-Model
health_cats = {
            0.:'Gesund',
            1.:'Varromilbenbefall',
            2.:'Varroamilbe',     
    }

interpreter = tf.lite.Interpreter(model_path='./varroa_health/model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
IMAGE_STRING_PLACEHOLDER = pil_to_b64(Image.open('uploads/test5.jpg').copy(), enc_format='jpeg')

# App-Layout
def serve_layout():
    return html.Div([

        # Banner display
        html.Div([
            html.H1(
                'Beehive Health Status',
                id='title'
            ),
            html.Img(className='twelve columns',
                     src='data:image/jpg;base64,{}'.format(encoded_image),
                     height='500px'
            )
        ],
            className="banner"
        ),

        # Body
        html.Div(className="twelve columns", children=[
            html.Div(className='row', children=[
                html.Div(
                    className='one columns',
                    style={'float': 'left'},
                    ),
                html.Div(className='four columns',style={'float': 'left'}, children=[
                    html.Br(),
                    html.Br(),
                    Card([
                        dcc.Upload(
                            id='upload-image',
                            children=[
                                'Drag and Drop or ',
                                html.A('Select an Image')
                            ],
                            style={
                                'width': '100%',
                                'height': '50px',
                                'lineHeight': '50px',
                                'borderWidth': '1px',
                                'borderStyle': 'dashed',
                                'borderRadius': '5px',
                                'textAlign': 'center'
                            },
                            accept='image/*'
                        ),    
                    ]),
                    dcc.Graph(id='original-img'),
                    
                ]),
            
                html.Div(
                    className='four columns',
                    style={'float': 'left'},
                    children=[
                        html.Br(),
                        html.Br(),
                        Card([
                        html.Div(children=[
                            html.H4('Predictions:')],
                            style={
                                'width': '100%',
                                'height': '50px',
                                'lineHeight': '50px',
                                'borderRadius': '5px',
                                'textAlign': 'center'
                            },)
                        ]),
                        dcc.Graph(id='new-img')
                        ]),
                html.Div(
                    className='two columns',
                    style={'float': 'left'},
                    children=[
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Div(className="u-full-width",id='table-content')
                        ]),
             #   html.Div(
             #       className='one columns',
             #       style={'float': 'left'},
             #       )
             #       ]
             #   ),
             #   
             #   html.Div(className='row', children=[
             #       html.Div(
             #       className='one columns',
             #       style={'float': 'left'},
             #       ),
             #       html.Div(
             #       className='five columns',
             #           children=[
             #               dcc.Graph(id='Temperature')
             #               ]),
             #       html.Div(className='row', children=[
             #       html.Div(
             #       className='five columns',
             #           children=[
             #               dcc.Graph(id='Humidity')
             #               ]),
             #       dcc.Interval(
             #           id='interval-component',
             #           interval=60*1000,
             #           n_intervals=0
             #           ),
             #       html.Div(
             #       className='one columns',
             #       style={'float': 'left'},
             #       )    
             #     ])          
                ]),
                html.Div(className='row', children=[
                    html.Br(),
                    html.Br(),
                    html.Br(),
                    html.Div(html.P("Credits: Banner was designed by macrovector / Freepik"))
                ])
            ])
        ])


app.layout = serve_layout


# Updating the Layout Output
@app.callback(Output('original-img', 'figure'),
              [Input('upload-image','contents'),
              Input('upload-image', 'filename')])
def original_img(contents, filename):
    # if image is uploaded the original image is ploted with this function
    if contents != None:
        string = contents.split(';base64,')[-1]
        im_pil = b64_to_pil(string)
    else:
        im_pil = b64_to_pil(IMAGE_STRING_PLACEHOLDER)
    figure = go.Figure()

    # Constants
    img_width = 900
    img_height = 500
    scale_factor = 0.5

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    figure.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    # Configure axes
    figure.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    figure.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    figure.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=im_pil)
    )

    # Configure other layout
    figure.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )
    return figure
   
    

@app.callback([Output('new-img', 'figure'),
               Output('table-content','children')],
              [Input('upload-image','contents')],
              [State('upload-image', 'filename')])
def new_img(contents, filename):
    # with this function the model is applied to the uploaded image and it is ploted. 
    if contents != None:
        string = contents.split(';base64,')[-1]
        im_pil = b64_to_pil(string)
        image = b64_to_numpy(string)
    else:
        im_pil = b64_to_pil(IMAGE_STRING_PLACEHOLDER)
    
    width, height = (im_pil.width, im_pil.height)
    im_resized = im_pil.resize((320, 320))
    new_image = np.array(im_resized)
    
    
    interpreter.set_tensor(input_details[0]['index'], [new_image])

    interpreter.invoke()
    rects = interpreter.get_tensor(
    output_details[0]['index'])
    scores = interpreter.get_tensor(
    output_details[2]['index'])

    health_class = interpreter.get_tensor(
    output_details[1]['index'])
    score_lst = []
    health_cat_lst = []
    for index, score in enumerate(scores[0]):
        if score > 0.5:
            score_lst.append( f'{score*100:.2f}%')
            health_cat_lst.append(health_cats[health_class[0][index]])
            draw_rect(new_image,rects[0][index])
    columns = ['Gesundheitszustand', 'Wahrscheinlichkeit']       
    df = pd.DataFrame(np.column_stack([health_cat_lst, score_lst]),
                      columns=columns)    
    final_img = Image.fromarray(new_image)
    
    
    figure = go.Figure()

    # Constants
    img_width = 900
    img_height = 500
    scale_factor = 0.5

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    figure.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    # Configure axes
    figure.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    figure.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    figure.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=final_img)
    )

    # Configure other layout
    figure.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )
    
    table = html.Div([
            dash_table.DataTable(
                data=df.to_dict('rows'),
                columns=[{'name': i, 'id': i} for i in df.columns]
            )
        ])
    
    return figure, table
#
#@app.callback([Output('Temperature', 'figure'),
#               Output('Humidity', 'figure')],
#              [Input('interval-component', 'n_intervals')]
#              )
#def plot_temp(n_intervals):
#    # Plot the Humidity and Temperature and updated the plot every n_intervals
#    temp_data = get_value(n_intervals)
#    
#    data = go.Scatter(
#            x=list(temp_data.dateandtime),
#            y=list(temp_data.temperature),
#            name='Scatter',
#            mode='lines+markers'
#            )
#    data1 = go.Scatter(
#            x=list(temp_data.dateandtime),
#            y=list(temp_data.humidity),
#            name='Scatter',
#            mode='lines+markers'
#            )
#    return {'data': [data], 
#            'layout' : go.Layout(title= 'Temperatur',xaxis=dict(title ='Zeit'),yaxis = dict(title ='Temperatur in °C'))}, {'data': [data1], 
#            'layout' : go.Layout(title= 'Luftfeuchtigkeit',xaxis=dict(title ='Zeit'),yaxis = dict(title ='Luftfeuchtigkeit in %'))} 
   
if __name__ == '__main__':
    app.run_server(port=8051)
    #app.run_server(host='0.0.0.0',port=8052)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2020 16:06:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:06:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 16:08:15] "POST /_dash-update-component